<a href="https://colab.research.google.com/github/licTomasPerez/Spin-Chains-/blob/main/expect_sxsx_sysy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
## las matrices sí están bien cargadas

from math import *
from cmath import *

N = 5
id2 = qeye(2)
sx = sigmax()
sy = sigmay()
sz = sigmaz()

sx_list = []
sy_list = []
sz_list = []

for n in range(N):
    operator_list = []
    for m in range(N):
        operator_list.append(id2)
    operator_list[n] = sx
    sx_list.append(tensor(operator_list))
        
    operator_list[n] = sy
    sy_list.append(tensor(operator_list))
        
    operator_list[n] = sz
    sz_list.append(tensor(operator_list))

sxsx_list = [sx_list[n]*sx_list[m] for n in range(N) for m in range(N)]
sysy_list = [sy_list[n]*sy_list[m] for n in range(N) for m in range(N)]

x_op = sum((.5 + sz_list[i])*i for i in range(N))

def W(a,b, k_max): 
    return sum(e**(1j * k * 2 * pi * (a-b) * N**-1)*(k*2*pi*N**-1) for k in range(k_max))

p_op = (sum(W(i,j,2)*(sxsx_list[i]*sxsx_list[j]+sysy_list[i]*sysy_list[j]) for i in range(N) for j in range(N)))

corr_xop_pop = [.5*(x_op[i] * p_op[i] + p_op[i] * x_op[i]) for i in range(N)]

print(corr_xop_pop)

[array([[188.49555922+0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j,   0.        +0.j,
          0.        +0.j,   0.        +0.j]]), array([[ 0.       +0.j, 87.9645943+0.j,  0.       +0.j,  0.       +0.j,
        -0.       +0.j,  0.       +0.j,  0.       +0.j,  0.       +0.j,
        -0.       +0.j,  0.       +0.j,  0.       +0.j,  0.       +0.j,
         0.       +0.j,  0.       +0.j,  0.       +0.j,  0.       +0.j,
         0.       +0.j,  0.       +0.j,  0. 

In [11]:
!pip install matplotlib
!pip install qutip

In [12]:
!pip install numpy

In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from math import * 
from cmath import * 
from qutip import *

In [28]:
### XYZ Heisenberg model

solver = "mc"   
N = 5            # número de spines

h  = 1.0 * 2 * np.pi * np.ones(N) 
Jz = 0.1 * 2 * np.pi * np.ones(N)
Jx = 0.15 * 2 * np.pi * np.ones(N)
Jy = 0.25 * 2 * np.pi * np.ones(N)

In [31]:
def hamiltonian(N, Jx, Jy, Jz, h, psi0, tlist, gamma, solver):
    
    id2 = qeye(2)
    sx = sigmax()
    sy = sigmay()
    sz = sigmaz()

    sx_list = []
    sy_list = []
    sz_list = []

    for n in range(N):
        operator_list = []
        for m in range(N):
            operator_list.append(id2)
        operator_list[n] = sx
        sx_list.append(tensor(operator_list))
        
        operator_list[n] = sy
        sy_list.append(tensor(operator_list))
        
        operator_list[n] = sz
        sz_list.append(tensor(operator_list))

    # la forma cool 
    sxsx_list = [sx_list[n]*sx_list[m] for n in range(N) for m in range(N)]
    sysy_list = [sy_list[n]*sy_list[m] for n in range(N) for m in range(N)]
    
    # no es terriblemente eficiente pero funca al menos
    # for n in range(N):
    #     for m in range(N):
    #         sxsx_list.append(sx_list[n]*sx_list[m])
    #         sysy_list.append(sy_list[n]*sy_list[m])

    x_op = sum((.5 + sz_list[i])*i for i in range(N))

    def W(a,b, k_max): 
        return sum(e**(1j * k * 2 * pi * (a-b) * N**-1)*(k*2*pi*N**-1) for k in range(k_max))

    p_op = (sum(W(i,j,2)*(sxsx_list[i]*sxsx_list[j]+sysy_list[i]*sysy_list[j]) for i in range(N) for j in range(N)))

    corr_xop_pop = [.5*(x_op[i] * p_op[i] + p_op[i] * x_op[i]) for i in range(N)]

    # Constructing the Hamiltonian
    H = 0
    
    for n in range(N):
        H += -0.5*h[n]*sz_list[n]
        
    for n in range(N-1):
        H += -0.5 * Jx[n] * sx_list[n] * sx_list[n+1]
        H += -0.5 * Jy[n] * sy_list[n] * sy_list[n+1]
        H += -0.5 * Jz[n] * sz_list[n] * sz_list[n+1]
    
    # collapse operators
    c_op_list = []

    # spin dephasing
    for n in range(N):
        if gamma[n] > 0.0:
            c_op_list.append(np.sqrt(gamma[n]) * sz_list[n])
    
    if solver == "me":
        result_spin_ops = mesolve(H, psi0, tlist, c_op_list, corr_xop_pop)
    elif solver == "mc":
        ntraj = 200 
        result_spin_ops = mcsolve(H, psi0, tlist, c_op_list, corr_xop_pop, ntraj)

    return result_spin_ops


In [32]:
# dephasing rate
gamma = 5 * 10**-10* np.ones(N)

# intial state, first spin in state |1>, the rest in state |0>
psi_list = []
for n in range(2):
    psi_list.append(basis(2,0))
psi_list.append(basis(2,1))
for n in range(2):
    psi_list.append(basis(2,0))
psi0 = tensor(psi_list)

tlist = np.linspace(0, 10, 100)

result_spin_ops = hamiltonian(N, Jx, Jy, Jz, h, psi0, tlist, gamma, solver)


TypeError: ignored

In [ ]:
plt.plot(np.real(result_spin_ops))

In [ ]:
def hamiltonian(N, Jx, Jy, Jz, h, psi0, tlist, gamma, solver):
    
    id2 = qeye(2)
    sx = sigmax()
    sy = sigmay()
    sz = sigmaz()

    sx_list = []
    sy_list = []
    sz_list = []

    for n in range(N):
        operator_list = []
        for m in range(N):
            operator_list.append(id2)
        operator_list[n] = sx
        sx_list.append(tensor(operator_list))
        
        operator_list[n] = sy
        sy_list.append(tensor(operator_list))
        
        operator_list[n] = sz
        sz_list.append(tensor(operator_list))

    sxsx_list = []
    sysy_list = []
    
    # no es terriblemente eficiente pero funca al menos
    for n in range(N):
      for m in range(N):
        sxsx_list.append(sx_list[n]*sx_list[n])
        sysy_list.append(sy_list[n]*sy_list[n])
    return sxsx_list, sysy_list

In [ ]:
# dephasing rate
gamma = 5 * 10**-10* np.ones(N)

# intial state, first spin in state |1>, the rest in state |0>
psi_list = []
for n in range(2):
    psi_list.append(basis(2,0))
psi_list.append(basis(2,1))
for n in range(2):
    psi_list.append(basis(2,0))
psi0 = tensor(psi_list)

tlist = np.linspace(0, 10, 100)

sXsX_list1, sYsY_list2 = hamiltonian(N, Jx, Jy, Jz, h, psi0, tlist, gamma, solver)

In [ ]:
sXsX_list1

[Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0

In [ ]:
sYsY_list2

[Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0